In [ ]:
import pandas
import sqlite3
import pandas as pd
from tqdm import tqdm
import os

In [2]:
# 化合物名 - 対象とするファイル
drugbank = "/workspace/99-NAS_data/pubmed/pubmed_drugbank_sentence/drugbank_sentences_v2.csv"

In [3]:
# 毒性名 - 対象とするファイル
hlgt = "/workspace/99-NAS_data/pubmed/pubmed_meddra_sentence/meddra_sentences_hlgt.csv"
hlt = "/workspace/99-NAS_data/pubmed/pubmed_meddra_sentence/meddra_sentences_hlt.csv"
llt = "/workspace/99-NAS_data/pubmed/pubmed_meddra_sentence/meddra_sentences_llt.csv"
pt = "/workspace/99-NAS_data/pubmed/pubmed_meddra_sentence/meddra_sentences_pt.csv"
soc = "/workspace/99-NAS_data/pubmed/pubmed_meddra_sentence/meddra_sentences_soc.csv"

In [4]:
df_drugbank  = pandas.read_csv(drugbank)

In [8]:
df_drugbank.head()

,drug_name,PMID,SENTID,SENTENCE
0,Lepirudin,10073268,36206825,The central role of thrombin generation in thi...
1,Lepirudin,10209835,36692541,HIT type II was presumed and recanalization wa...
2,Lepirudin,10209835,36692547,This case report illustrates the complicated d...
3,Lepirudin,10218429,36748792,The patient was treated with lepirudin at body...
4,Lepirudin,10229643,36825405,"Lepirudin has a short half-life, and only 50-6..."


In [13]:
print(len(df_drugbank))
sentid_drug = df_drugbank['SENTID'].tolist()
print(min(sentid_drug), max(sentid_drug))

40256637
2 232293247


In [10]:
df_hlgt  = pandas.read_csv(hlgt)

In [12]:
df_hlgt.head()

,level,term,PMID,SENTID,SENTENCE
0,hlgt,Abortions and stillbirth,10050933,36125580,Although the number of families so far ascerta...
1,hlgt,Abortions and stillbirth,10221883,36773822,"To determine whether Neospora caninum, a paras..."
2,hlgt,Abortions and stillbirth,10457505,37810488,This study investigates pregnancy loss associa...
3,hlgt,Acid-base disorders,10150759,36474756,"However, factors unrelated to splanchnic ische..."
4,hlgt,Acid-base disorders,10233126,36847981,The physicochemical origins for this acid-base...


In [14]:
print(len(df_hlgt))
sentid_hlgt = df_hlgt['SENTID'].tolist()
print(min(sentid_hlgt), max(sentid_hlgt))

2117668
1260 232292886


In [15]:
overlap_sentid = set(sentid_drug) & set(sentid_hlgt)
print("Overlap SENTID count:", len(overlap_sentid))

Overlap SENTID count: 166828


In [ ]:
conn = sqlite3.connect("merge_work.db")

for chunk in tqdm(pd.read_csv(drugbank, chunksize=100000)):
    chunk.to_sql("drug_table", conn, if_exists="append", index=False)

for chunk in tqdm(pd.read_csv(hlgt, chunksize=100000)):
    chunk.to_sql("hlgt_table", conn, if_exists="append", index=False)

conn.execute("CREATE INDEX idx_drug ON drug_table(PMID, SENTID)")
conn.execute("CREATE INDEX idx_hlgt ON hlgt_table(PMID, SENTID)")

query = """
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.drug_name,
    m.term,
    m.level
FROM
    drug_table d
INNER JOIN
    hlgt_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
"""

for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=100000)):
    chunk.to_csv("co_occurrence_sentences_drugbank_hlgt.csv", mode='a', index=False, header=not os.path.exists("co_occurrence_sentences_drugbank_hlgt.csv"))

conn.close()
print("完了しました。")

3it [00:39, 13.00s/it]

完了しました。


In [20]:
conn = sqlite3.connect("/workspace/99-NAS_data/pubmed/merged_database/merge_work.db")

for chunk in tqdm(pd.read_csv(hlt, chunksize=100000)):
    chunk.to_sql("hlt_table", conn, if_exists="append", index=False)

conn.execute("CREATE INDEX idx_hlt ON hlt_table(PMID, SENTID)")

query = """
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.drug_name,
    m.term,
    m.level
FROM
    drug_table d
INNER JOIN
    hlt_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
"""

for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=100000)):
    chunk.to_csv("/workspace/99-NAS_data/pubmed/merged_database/co_occurrence_sentences_drugbank_hlt.csv", mode='a', index=False, header=not os.path.exists("/workspace/99-NAS_data/pubmed/merged_database/co_occurrence_sentences_drugbank_hlt.csv"))

conn.close()
print("完了しました。")

99it [01:31,  1.08it/s]
24it [01:25,  3.57s/it]

完了しました。


In [21]:
conn = sqlite3.connect("/workspace/99-NAS_data/pubmed/merged_database/merge_work.db")

for chunk in tqdm(pd.read_csv(llt, chunksize=100000)):
    chunk.to_sql("llt_table", conn, if_exists="append", index=False)

conn.execute("CREATE INDEX idx_llt ON llt_table(PMID, SENTID)")

query = """
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.drug_name,
    m.term,
    m.level
FROM
    drug_table d
INNER JOIN
    llt_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
"""

for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=100000)):
    chunk.to_csv("/workspace/99-NAS_data/pubmed/merged_database/co_occurrence_sentences_drugbank_llt.csv", mode='a', index=False, header=not os.path.exists("/workspace/99-NAS_data/pubmed/merged_database/co_occurrence_sentences_drugbank_llt.csv"))

conn.close()
print("完了しました。")

3418it [36:40,  1.55it/s]
806it [2:18:25, 10.30s/it]

完了しました。


In [22]:
conn = sqlite3.connect("/workspace/99-NAS_data/pubmed/merged_database/merge_work.db")

for chunk in tqdm(pd.read_csv(pt, chunksize=100000)):
    chunk.to_sql("pt_table", conn, if_exists="append", index=False)

conn.execute("CREATE INDEX idx_pt ON pt_table(PMID, SENTID)")

query = """
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.drug_name,
    m.term,
    m.level
FROM
    drug_table d
INNER JOIN
    pt_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
"""

for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=100000)):
    chunk.to_csv("/workspace/99-NAS_data/pubmed/merged_database/co_occurrence_sentences_drugbank_pt.csv", mode='a', index=False, header=not os.path.exists("/workspace/99-NAS_data/pubmed/merged_database/co_occurrence_sentences_drugbank_pt.csv"))

conn.close()
print("完了しました。")

1368it [14:43,  1.55it/s]
266it [21:24,  4.83s/it]

完了しました。


In [23]:
conn = sqlite3.connect("/workspace/99-NAS_data/pubmed/merged_database/merge_work.db")

for chunk in tqdm(pd.read_csv(soc, chunksize=100000)):
    chunk.to_sql("soc_table", conn, if_exists="append", index=False)

conn.execute("CREATE INDEX idx_soc ON soc_table(PMID, SENTID)")

query = """
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.drug_name,
    m.term,
    m.level
FROM
    drug_table d
INNER JOIN
    soc_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
"""

for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=100000)):
    chunk.to_csv("/workspace/99-NAS_data/pubmed/merged_database/co_occurrence_sentences_drugbank_soc.csv", mode='a', index=False, header=not os.path.exists("/workspace/99-NAS_data/pubmed/merged_database/co_occurrence_sentences_drugbank_soc.csv"))

conn.close()
print("完了しました。")

55it [00:34,  1.59it/s]
14it [00:53,  3.80s/it]

完了しました。


# watch

In [24]:
import pandas as pd
df = pd.read_csv("/workspace/99-NAS_data/pubmed/merged_database/co_occurrence_sentences_drugbank_hlgt.csv")

In [25]:
df.head()

,PMID,SENTID,SENTENCE,drug_name,term,level
0,11091212,41827744,The anti-CD33 monoclonal antibody linked to NA...,Gemtuzumab ozogamicin,Leukaemias,hlgt
1,11520253,44486823,Pegademase is used in the treatment of severe ...,Pegademase,Leukaemias,hlgt
2,10087987,36297098,The ability of low-dose high frequency algluce...,Alglucerase,Fractures,hlgt
3,10604708,38763363,Logistic regression analysis revealed that imm...,Capromab pendetide,Metastases,hlgt
4,10604708,38763365,Immunoscintigraphy with 111In-capromab pendeti...,Capromab pendetide,Metastases,hlgt


In [26]:
df.iloc[0]

PMID                                                  11091212
SENTID                                                41827744
SENTENCE     The anti-CD33 monoclonal antibody linked to NA...
drug_name                                Gemtuzumab ozogamicin
term                                                Leukaemias
level                                                     hlgt
Name: 0, dtype: object

# pubchem × meddra

In [28]:
pubchem = "/workspace/99-NAS_data/pubmed/pubmed_pubchem_sentence/pubchem_sentences.csv"

In [ ]:
conn = sqlite3.connect("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/merge_work.db")

# for chunk in tqdm(pd.read_csv(pubchem, chunksize=100000, engine='python')):
#     chunk.to_sql("chemical_table", conn, if_exists="append", index=False)

# for chunk in tqdm(pd.read_csv(hlgt, chunksize=100000)):
#     chunk.to_sql("hlgt_table", conn, if_exists="append", index=False)

# conn.execute("CREATE INDEX idx_chemical ON chemical_table(PMID, SENTID)")
# conn.execute("CREATE INDEX idx_hlgt ON hlgt_table(PMID, SENTID)")

query = """
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.TERM,
    m.term,
    m.level
FROM
    chemical_table d
INNER JOIN
    hlgt_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
"""

for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=100000)):
    chunk.to_csv("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/co_occurrence_sentences_chemical_hlgt.csv", mode='a', index=False, header=not os.path.exists("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/co_occurrence_sentences_chemical_hlgt.csv"))

conn.close()
print("完了しました。")

DatabaseError: Execution failed on sql '
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.TERM,
    m.term,
    m.level
FROM
    chemical_table d
INNER JOIN
    hlgt_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
': no such column: d.TERM

In [33]:
print(pd.read_sql_query("PRAGMA table_info(chemical_table)", conn))
print(pd.read_sql_query("PRAGMA table_info(hlgt_table)", conn))

   cid      name     type  notnull dflt_value  pk
0    0      PMID  INTEGER        0       None   0
1    1    SENTID  INTEGER        0       None   0
2    2  SENTENCE     TEXT        0       None   0
   cid      name     type  notnull dflt_value  pk
0    0     level     TEXT        0       None   0
1    1      term     TEXT        0       None   0
2    2      PMID  INTEGER        0       None   0
3    3    SENTID  INTEGER        0       None   0
4    4  SENTENCE     TEXT        0       None   0


In [37]:
import pandas as pd
print(pd.read_csv(pubchem, nrows=5, engine='python'))

                  PMID     SENTID  \
NCE                                 
Phoxim, (E)-  32284137  169740915   
metabolitie    1415972    3904317   
EC23          19381361   80268633   
EC23          19381361   80268637   
EC23          19381361   80268638   

                                                       SENTENCE  
NCE                                                              
Phoxim, (E)-  The body weight and survival rate of the phoxi...  
metabolitie   Analgesic efficacy was assessed by a nurse-obs...  
EC23          We present the novel application of proteomic ...  
EC23          The cellular retinoid-responsive proteins CRAB...  
EC23          In addition, the actin filament regulatory pro...  


In [ ]:
conn = sqlite3.connect("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/merge_work.db")

for chunk in tqdm(pd.read_csv(hlgt, chunksize=100000)):
    chunk.to_sql("hlt_table", conn, if_exists="append", index=False)

conn.execute("CREATE INDEX idx_chemical ON chemical_table(PMID, SENTID)")
conn.execute("CREATE INDEX idx_hlt ON hlt_table(PMID, SENTID)")

query = """
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.TERM,
    m.term,
    m.level
FROM
    chemical_table d
INNER JOIN
    hlt_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
"""

for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=100000)):
    chunk.to_csv("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/co_occurrence_sentences_chemical_hlt.csv", mode='a', index=False, header=not os.path.exists("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/co_occurrence_sentences_chemical_hlt.csv"))

conn.close()
print("完了しました。")

In [ ]:
conn = sqlite3.connect("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/merge_work.db")

for chunk in tqdm(pd.read_csv(llt, chunksize=100000)):
    chunk.to_sql("llt_table", conn, if_exists="append", index=False)

conn.execute("CREATE INDEX idx_chemical ON chemical_table(PMID, SENTID)")
conn.execute("CREATE INDEX idx_hlt ON llt_table(PMID, SENTID)")

query = """
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.TERM,
    m.term,
    m.level
FROM
    chemical_table d
INNER JOIN
    llt_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
"""

for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=100000)):
    chunk.to_csv("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/co_occurrence_sentences_chemical_llt.csv", mode='a', index=False, header=not os.path.exists("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/co_occurrence_sentences_chemical_llt.csv"))

conn.close()
print("完了しました。")

In [ ]:
conn = sqlite3.connect("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/merge_work.db")

for chunk in tqdm(pd.read_csv(pt, chunksize=100000)):
    chunk.to_sql("pt_table", conn, if_exists="append", index=False)

conn.execute("CREATE INDEX idx_chemical ON chemical_table(PMID, SENTID)")
conn.execute("CREATE INDEX idx_pt ON pt_table(PMID, SENTID)")

query = """
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.TERM,
    m.term,
    m.level
FROM
    chemical_table d
INNER JOIN
    pt_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
"""

for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=100000)):
    chunk.to_csv("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/co_occurrence_sentences_chemical_pt.csv", mode='a', index=False, header=not os.path.exists("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/co_occurrence_sentences_chemical_pt.csv"))

conn.close()
print("完了しました。")

In [ ]:
conn = sqlite3.connect("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/merge_work.db")

for chunk in tqdm(pd.read_csv(soc, chunksize=100000)):
    chunk.to_sql("soc_table", conn, if_exists="append", index=False)

conn.execute("CREATE INDEX idx_chemical ON chemical_table(PMID, SENTID)")
conn.execute("CREATE INDEX idx_soc ON soc_table(PMID, SENTID)")

query = """
SELECT
    d.PMID,
    d.SENTID,
    d.SENTENCE,
    d.TERM,
    m.term,
    m.level
FROM
    chemical_table d
INNER JOIN
    soc_table m
ON
    d.PMID = m.PMID AND d.SENTID = m.SENTID
"""

for chunk in tqdm(pd.read_sql_query(query, conn, chunksize=100000)):
    chunk.to_csv("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/co_occurrence_sentences_chemical_soc.csv", mode='a', index=False, header=not os.path.exists("/workspace/99-NAS_data/pubmed/pubchem_meddra_merged_database/co_occurrence_sentences_chemical_soc.csv"))

conn.close()
print("完了しました。")